In [1]:
# Bag of apps categories
# Bag of labels categories
# Include phone brand and model device

print("Initialize libraries")

import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.cross_validation import StratifiedKFold, KFold
from sklearn.metrics import log_loss
from sklearn.cluster import DBSCAN
from sklearn import metrics as skmetrics
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from collections import Counter
from keras.layers.advanced_activations import PReLU
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import ensemble
from sklearn.decomposition import PCA
import os
import gc
from scipy import sparse
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, SelectKBest
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

Initialize libraries


Using Theano backend.


In [2]:
#------------------------------------------------- Write functions ----------------------------------------

def rstr(df): return df.dtypes, df.head(3) ,df.apply(lambda x: [x.unique()]), df.apply(lambda x: [len(x.unique())]),df.shape

def batch_generator(X, y, batch_size, shuffle):
    #chenglong code for fiting from generator (https://www.kaggle.com/c/talkingdata-mobile-user-demographics/forums/t/22567/neural-network-for-sparse-matrices)
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

In [3]:

#------------------------------------------------ Read data from source files ------------------------------------

seed = 7
np.random.seed(seed)
datadir = ''

print("### ----- PART 1 ----- ###")

# Data - Events data
# Bag of apps
print("# Read app events")
app_events = pd.read_csv(os.path.join(datadir,'app_events.csv'), dtype={'device_id' : np.str})
app_events.head(5)
app_events.info()
#print(rstr(app_events))

# remove duplicates(app_id)
app_events= app_events.groupby("event_id")["app_id"].apply(
    lambda x: " ".join(set("app_id:" + str(s) for s in x)))
app_events.head(5)

print("# Read Events")
events = pd.read_csv(os.path.join(datadir,'events.csv'), dtype={'device_id': np.str})
events.head(5)
events["app_id"] = events["event_id"].map(app_events)
events = events.dropna()
del app_events

events = events[["device_id", "app_id"]]
events.info()
# 1Gb reduced to 34 Mb

# remove duplicates(app_id)
events.loc[:,"device_id"].value_counts(ascending=True)

events = events.groupby("device_id")["app_id"].apply(
    lambda x: " ".join(set(str(" ".join(str(s) for s in x)).split(" "))))
events = events.reset_index(name="app_id")

# expand to multiple rows
events = pd.concat([pd.Series(row['device_id'], row['app_id'].split(' '))
                    for _, row in events.iterrows()]).reset_index()
events.columns = ['app_id', 'device_id']
events.head(5)
f3 = events[["device_id", "app_id"]]    # app_id

print("#Part1 formed")

### ----- PART 1 ----- ###
# Read app events
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32473067 entries, 0 to 32473066
Data columns (total 4 columns):
event_id        int64
app_id          int64
is_installed    int64
is_active       int64
dtypes: int64(4)
memory usage: 991.0 MB
# Read Events
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1488096 entries, 1 to 3252947
Data columns (total 2 columns):
device_id    1488096 non-null object
app_id       1488096 non-null object
dtypes: object(2)
memory usage: 34.1+ MB
#Part1 formed


In [4]:
##################
#   App labels
##################


print("### ----- PART 2 ----- ###")

print("# Read App labels")
app_labels = pd.read_csv(os.path.join(datadir,'app_labels.csv'))
label_cat = pd.read_csv(os.path.join(datadir,'label_categories.csv'))
app_labels.info()
label_cat.info()
label_cat=label_cat[['label_id','category']]

app_labels=app_labels.merge(label_cat,on='label_id',how='left')
app_labels.head(3)
events.head(3)
#app_labels = app_labels.loc[app_labels.smaller_cat != "unknown_unknown"]

#app_labels = app_labels.groupby("app_id")["category"].apply(
#    lambda x: ";".join(set("app_cat:" + str(s) for s in x)))
app_labels = app_labels.groupby(["app_id","category"]).agg('size').reset_index()
app_labels = app_labels[['app_id','category']]
print("# App labels done")


# Remove "app_id:" from column
print("## Handling events data for merging with app lables")
events['app_id'] = events['app_id'].map(lambda x : x.lstrip('app_id:'))
events['app_id'] = events['app_id'].astype(str)
app_labels['app_id'] = app_labels['app_id'].astype(str)
app_labels.info()

print("## Merge")

events= pd.merge(events, app_labels, on = 'app_id',how='left').astype(str)
#events['smaller_cat'].unique()

# expand to multiple rows
print("#Expand to multiple rows")
#events= pd.concat([pd.Series(row['device_id'], row['category'].split(';'))
#                    for _, row in events.iterrows()]).reset_index()
#events.columns = ['app_cat', 'device_id']
#events.head(5)
#print(events.info())

events= events.groupby(["device_id","category"]).agg('size').reset_index()
events= events[['device_id','category']]
events.head(10)
print("# App labels done")

f5 = events[["device_id", "category"]]    # app_id
# Can % total share be included as well?
print("# App category part formed")

### ----- PART 2 ----- ###
# Read App labels
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459943 entries, 0 to 459942
Data columns (total 2 columns):
app_id      459943 non-null int64
label_id    459943 non-null int64
dtypes: int64(2)
memory usage: 7.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 2 columns):
label_id    930 non-null int64
category    927 non-null object
dtypes: int64(1), object(1)
memory usage: 14.6+ KB
# App labels done
## Handling events data for merging with app lables
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 457777 entries, 0 to 457776
Data columns (total 2 columns):
app_id      457777 non-null object
category    457777 non-null object
dtypes: object(2)
memory usage: 7.0+ MB
## Merge
#Expand to multiple rows
# App labels done
# App category part formed


In [5]:
##################
#   Phone Brand
##################
print("### ----- PART 3 ----- ###")

print("# Read Phone Brand")
pbd = pd.read_csv(os.path.join(datadir,'phone_brand_device_model.csv'),
                  dtype={'device_id': np.str})
pbd.drop_duplicates('device_id', keep='first', inplace=True)

##################
#  Train and Test
##################
print("# Generate Train and Test")

train = pd.read_csv(os.path.join(datadir,'gender_age_train.csv'),
                    dtype={'device_id': np.str})
train.drop(["age", "gender"], axis=1, inplace=True)

test = pd.read_csv(os.path.join(datadir,'gender_age_test.csv'),
                   dtype={'device_id': np.str})
test["group"] = np.nan


split_len = len(train)

# Group Labels
Y = train["group"]
lable_group = LabelEncoder()
Y = lable_group.fit_transform(Y)
device_id = test["device_id"]

# Concat
Df = pd.concat((train, test), axis=0, ignore_index=True)

print("### ----- PART 4 ----- ###")

Df = pd.merge(Df, pbd, how="left", on="device_id")
Df["phone_brand"] = Df["phone_brand"].apply(lambda x: "phone_brand:" + str(x))
Df["device_model"] = Df["device_model"].apply(
    lambda x: "device_model:" + str(x))

### ----- PART 3 ----- ###
# Read Phone Brand
# Generate Train and Test
### ----- PART 4 ----- ###


In [6]:
###################
#  Concat Feature
###################

print("# Concat all features")

f1 = Df[["device_id", "phone_brand"]]   # phone_brand
f2 = Df[["device_id", "device_model"]]  # device_model

events = None
Df = None

f1.columns.values[1] = "feature"
f2.columns.values[1] = "feature"
f5.columns.values[1] = "feature"
f3.columns.values[1] = "feature"

FLS = pd.concat((f1, f2, f3, f5), axis=0, ignore_index=True)

FLS.info()

###################
# User-Item Feature
###################
print("# User-Item-Feature")

device_ids = FLS["device_id"].unique()
feature_cs = FLS["feature"].unique()

data = np.ones(len(FLS))
len(data)

dec = LabelEncoder().fit(FLS["device_id"])
row = dec.transform(FLS["device_id"])
col = LabelEncoder().fit_transform(FLS["feature"])
sparse_matrix = sparse.csr_matrix(
    (data, (row, col)), shape=(len(device_ids), len(feature_cs)))
sparse_matrix.shape
sys.getsizeof(sparse_matrix)

sparse_matrix = sparse_matrix[:, sparse_matrix.getnnz(0) > 0]
print("# Sparse matrix done")

del FLS
del data
f1 = [1]
f5 = [1]
f2 = [1]
f3 = [1]

events = [1]

# Concat all features
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6677856 entries, 0 to 6677855
Data columns (total 2 columns):
device_id    object
feature      object
dtypes: object(2)
memory usage: 101.9+ MB
# User-Item-Feature
# Sparse matrix done


In [7]:
##################
#      Data
##################

print("# Split data")
train_row = dec.transform(train["device_id"])
train_sp = sparse_matrix[train_row, :]

test_row = dec.transform(test["device_id"])
test_sp = sparse_matrix[test_row, :]

X_train, X_val, y_train, y_val = train_test_split(
    train_sp, Y, train_size=0.999, random_state=10)

##################
#   Feature Sel
##################
print("# Feature Selection")
#selector = SelectPercentile(f_classif, percentile=53)

#selector.fit(X_train, y_train)
#X_train.shape
#X_train = selector.transform(X_train)
#X_train.shape
#X_val = selector.transform(X_val)
#X_val.shape

# Selection using chi-square
# selector = SelectKBest(chi2, k=11155).fit(X_train, y_train)
# X_train.shape
# X_train = selector.transform(X_train)
# X_train.shape
# X_val = selector.transform(X_val)
# X_val.shape

print("# Num of Features: ", X_train.shape[1])

# Split data
# Feature Selection
('# Num of Features: ', 21425)


In [8]:
##################
#  Build Model
##################


#act = keras.layers.advanced_activations.PReLU(init='zero', weights=None)

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(150, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.5))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model


model = baseline_model()

fit= model.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69984,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )


# evaluate the model
scores_val = model.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val)))


#print("# Final prediction")
#scores = model.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
#result = pd.DataFrame(scores , columns=lable_group.classes_)
#result["device_id"] = device_id
#print(result.head(1))
#result = result.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result.to_csv('keras_' + str(log_loss(y_val, scores_val)) + '.csv', index=True, index_label='device_id')


print("Done")

Epoch 1/18


/home/ubuntu/anaconda/lib/python2.7/site-packages/keras/engine/training.py:1463: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


26s - loss: 2.4395 - acc: 0.1309 - val_loss: 2.4228 - val_acc: 0.0933
Epoch 2/18
26s - loss: 2.3798 - acc: 0.1538 - val_loss: 2.3867 - val_acc: 0.0800
Epoch 3/18
26s - loss: 2.3435 - acc: 0.1661 - val_loss: 2.3622 - val_acc: 0.0933
Epoch 4/18
26s - loss: 2.3225 - acc: 0.1762 - val_loss: 2.3400 - val_acc: 0.0667
Epoch 5/18
26s - loss: 2.3071 - acc: 0.1800 - val_loss: 2.3202 - val_acc: 0.0667
Epoch 6/18
26s - loss: 2.2933 - acc: 0.1867 - val_loss: 2.3088 - val_acc: 0.0933
Epoch 7/18
26s - loss: 2.2849 - acc: 0.1875 - val_loss: 2.3112 - val_acc: 0.1067
Epoch 8/18
26s - loss: 2.2761 - acc: 0.1932 - val_loss: 2.3037 - val_acc: 0.0800
Epoch 9/18
26s - loss: 2.2668 - acc: 0.1983 - val_loss: 2.2981 - val_acc: 0.1067
Epoch 10/18
26s - loss: 2.2624 - acc: 0.1979 - val_loss: 2.3052 - val_acc: 0.1067
Epoch 11/18
26s - loss: 2.2537 - acc: 0.2029 - val_loss: 2.2916 - val_acc: 0.0800
Epoch 12/18
26s - loss: 2.2489 - acc: 0.2033 - val_loss: 2.2939 - val_acc: 0.1067
Epoch 13/18
26s - loss: 2.2459 - acc

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [9]:
def second_model():
    # create model
    model = Sequential()
    model.add(Dense(155, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_2 = second_model()

fit_2 = model_2.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69784,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

scores_val_2 = model_2.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])

Epoch 1/18
27s - loss: 2.4271 - acc: 0.1384 - val_loss: 2.4214 - val_acc: 0.0667
Epoch 2/18
27s - loss: 2.3521 - acc: 0.1631 - val_loss: 2.3485 - val_acc: 0.0667
Epoch 3/18
27s - loss: 2.3149 - acc: 0.1762 - val_loss: 2.3236 - val_acc: 0.0800
Epoch 4/18
27s - loss: 2.2955 - acc: 0.1844 - val_loss: 2.3007 - val_acc: 0.0800
Epoch 5/18
27s - loss: 2.2847 - acc: 0.1900 - val_loss: 2.2972 - val_acc: 0.1200
Epoch 6/18
27s - loss: 2.2703 - acc: 0.1950 - val_loss: 2.2851 - val_acc: 0.0800
Epoch 7/18
27s - loss: 2.2622 - acc: 0.1978 - val_loss: 2.2844 - val_acc: 0.1200
Epoch 8/18
27s - loss: 2.2578 - acc: 0.2014 - val_loss: 2.2887 - val_acc: 0.0933
Epoch 9/18
27s - loss: 2.2467 - acc: 0.2059 - val_loss: 2.2736 - val_acc: 0.1600
Epoch 10/18
27s - loss: 2.2403 - acc: 0.2064 - val_loss: 2.2706 - val_acc: 0.1867
Epoch 11/18
27s - loss: 2.2354 - acc: 0.2097 - val_loss: 2.2759 - val_acc: 0.1467
Epoch 12/18
27s - loss: 2.2294 - acc: 0.2116 - val_loss: 2.2725 - val_acc: 0.1600
Epoch 13/18
27s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [10]:
def third_model():
    # create model
    model = Sequential()
    model.add(Dense(145, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_3 = third_model()

fit_3 = model_3.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

scores_val_3 = model_3.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_3)))

Epoch 1/18
24s - loss: 2.4297 - acc: 0.1383 - val_loss: 2.4198 - val_acc: 0.0667
Epoch 2/18
24s - loss: 2.3601 - acc: 0.1620 - val_loss: 2.3775 - val_acc: 0.0800
Epoch 3/18
24s - loss: 2.3169 - acc: 0.1778 - val_loss: 2.3454 - val_acc: 0.0933
Epoch 4/18
24s - loss: 2.2998 - acc: 0.1833 - val_loss: 2.3260 - val_acc: 0.1333
Epoch 5/18
24s - loss: 2.2871 - acc: 0.1882 - val_loss: 2.3225 - val_acc: 0.0933
Epoch 6/18
24s - loss: 2.2727 - acc: 0.1940 - val_loss: 2.3194 - val_acc: 0.1067
Epoch 7/18
24s - loss: 2.2680 - acc: 0.1965 - val_loss: 2.2938 - val_acc: 0.1200
Epoch 8/18
18s - loss: 2.2561 - acc: 0.2016 - val_loss: 2.3034 - val_acc: 0.1467
Epoch 9/18
18s - loss: 2.2532 - acc: 0.2013 - val_loss: 2.2960 - val_acc: 0.1600
Epoch 10/18
18s - loss: 2.2388 - acc: 0.2057 - val_loss: 2.2899 - val_acc: 0.1733
Epoch 11/18
18s - loss: 2.2383 - acc: 0.2083 - val_loss: 2.2917 - val_acc: 0.1333
Epoch 12/18
18s - loss: 2.2297 - acc: 0.2127 - val_loss: 2.3073 - val_acc: 0.0933
Epoch 13/18
18s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [11]:
def forth_model():
    # create model
    model = Sequential()
    model.add(Dense(147, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_4 = forth_model()

fit_4 = model_4.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69984,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_4 = model_4.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_4)))

Epoch 1/18
26s - loss: 2.4287 - acc: 0.1397 - val_loss: 2.4274 - val_acc: 0.0667
Epoch 2/18
26s - loss: 2.3508 - acc: 0.1660 - val_loss: 2.3647 - val_acc: 0.0933
Epoch 3/18
26s - loss: 2.3132 - acc: 0.1782 - val_loss: 2.3385 - val_acc: 0.0800
Epoch 4/18
26s - loss: 2.2929 - acc: 0.1853 - val_loss: 2.3238 - val_acc: 0.0933
Epoch 5/18
26s - loss: 2.2809 - acc: 0.1910 - val_loss: 2.3257 - val_acc: 0.0667
Epoch 6/18
26s - loss: 2.2726 - acc: 0.1952 - val_loss: 2.3175 - val_acc: 0.0667
Epoch 7/18
26s - loss: 2.2639 - acc: 0.1956 - val_loss: 2.3141 - val_acc: 0.0933
Epoch 8/18
26s - loss: 2.2529 - acc: 0.2006 - val_loss: 2.2924 - val_acc: 0.1467
Epoch 9/18
26s - loss: 2.2485 - acc: 0.2025 - val_loss: 2.3019 - val_acc: 0.1067
Epoch 10/18
26s - loss: 2.2395 - acc: 0.2082 - val_loss: 2.2989 - val_acc: 0.1333
Epoch 11/18
26s - loss: 2.2357 - acc: 0.2069 - val_loss: 2.3004 - val_acc: 0.1067
Epoch 12/18
26s - loss: 2.2295 - acc: 0.2107 - val_loss: 2.3055 - val_acc: 0.1067
Epoch 13/18
26s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [12]:
def fifth_model():
    # create model
    model = Sequential()
    model.add(Dense(152, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.42))
    model.add(Dense(52, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.18))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_5 = fifth_model()

fit_5 = model_5.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_5 = model_5.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_5)))

Epoch 1/18
26s - loss: 2.4315 - acc: 0.1362 - val_loss: 2.4236 - val_acc: 0.0667
Epoch 2/18
26s - loss: 2.3562 - acc: 0.1631 - val_loss: 2.3699 - val_acc: 0.0800
Epoch 3/18
26s - loss: 2.3170 - acc: 0.1770 - val_loss: 2.3345 - val_acc: 0.0800
Epoch 4/18
26s - loss: 2.2994 - acc: 0.1855 - val_loss: 2.3408 - val_acc: 0.0400
Epoch 5/18
26s - loss: 2.2847 - acc: 0.1891 - val_loss: 2.3274 - val_acc: 0.0533
Epoch 6/18
26s - loss: 2.2760 - acc: 0.1934 - val_loss: 2.3113 - val_acc: 0.0533
Epoch 7/18
26s - loss: 2.2626 - acc: 0.1983 - val_loss: 2.3081 - val_acc: 0.0533
Epoch 8/18
26s - loss: 2.2589 - acc: 0.2002 - val_loss: 2.3138 - val_acc: 0.0800
Epoch 9/18
26s - loss: 2.2493 - acc: 0.2047 - val_loss: 2.3080 - val_acc: 0.1333
Epoch 10/18
26s - loss: 2.2442 - acc: 0.2072 - val_loss: 2.3105 - val_acc: 0.1333
Epoch 11/18
26s - loss: 2.2388 - acc: 0.2088 - val_loss: 2.3041 - val_acc: 0.1600
Epoch 12/18
26s - loss: 2.2287 - acc: 0.2088 - val_loss: 2.2971 - val_acc: 0.1867
Epoch 13/18
26s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [13]:
def sixth_model():
    # create model
    model = Sequential()
    model.add(Dense(153, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.42))
    model.add(Dense(51, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_6 = fifth_model()

fit_6 = model_6.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_6 = model_6.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_5)))

Epoch 1/18
26s - loss: 2.4257 - acc: 0.1368 - val_loss: 2.4218 - val_acc: 0.0933
Epoch 2/18
26s - loss: 2.3514 - acc: 0.1656 - val_loss: 2.3680 - val_acc: 0.0667
Epoch 3/18
26s - loss: 2.3176 - acc: 0.1748 - val_loss: 2.3355 - val_acc: 0.0800
Epoch 4/18
26s - loss: 2.2992 - acc: 0.1812 - val_loss: 2.3074 - val_acc: 0.0933
Epoch 5/18
26s - loss: 2.2815 - acc: 0.1887 - val_loss: 2.2989 - val_acc: 0.1067
Epoch 6/18
26s - loss: 2.2747 - acc: 0.1927 - val_loss: 2.2978 - val_acc: 0.1200
Epoch 7/18
26s - loss: 2.2652 - acc: 0.1967 - val_loss: 2.2945 - val_acc: 0.1067
Epoch 8/18
26s - loss: 2.2563 - acc: 0.2010 - val_loss: 2.2986 - val_acc: 0.1200
Epoch 9/18
26s - loss: 2.2542 - acc: 0.2016 - val_loss: 2.3029 - val_acc: 0.1333
Epoch 10/18
26s - loss: 2.2394 - acc: 0.2073 - val_loss: 2.2894 - val_acc: 0.1600
Epoch 11/18
26s - loss: 2.2365 - acc: 0.2082 - val_loss: 2.2921 - val_acc: 0.1600
Epoch 12/18
26s - loss: 2.2315 - acc: 0.2100 - val_loss: 2.2965 - val_acc: 0.1333
Epoch 13/18
26s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [14]:
def seventh_model():
    # create model
    model = Sequential()
    model.add(Dense(144, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(50, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_7 = seventh_model()

fit_7 = model_7.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_7 = model_7.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_7)))

Epoch 1/18
25s - loss: 2.4311 - acc: 0.1344 - val_loss: 2.4119 - val_acc: 0.0667
Epoch 2/18
25s - loss: 2.3571 - acc: 0.1616 - val_loss: 2.3568 - val_acc: 0.1200
Epoch 3/18
25s - loss: 2.3182 - acc: 0.1759 - val_loss: 2.3274 - val_acc: 0.1067
Epoch 4/18
25s - loss: 2.2968 - acc: 0.1836 - val_loss: 2.3094 - val_acc: 0.0933
Epoch 5/18
25s - loss: 2.2837 - acc: 0.1882 - val_loss: 2.3049 - val_acc: 0.0933
Epoch 6/18
25s - loss: 2.2736 - acc: 0.1935 - val_loss: 2.2914 - val_acc: 0.1467
Epoch 7/18
25s - loss: 2.2618 - acc: 0.1966 - val_loss: 2.3033 - val_acc: 0.1200
Epoch 8/18
25s - loss: 2.2570 - acc: 0.1996 - val_loss: 2.2930 - val_acc: 0.1467
Epoch 9/18
25s - loss: 2.2499 - acc: 0.2025 - val_loss: 2.2907 - val_acc: 0.1467
Epoch 10/18
25s - loss: 2.2406 - acc: 0.2081 - val_loss: 2.2766 - val_acc: 0.1467
Epoch 11/18
25s - loss: 2.2338 - acc: 0.2103 - val_loss: 2.2826 - val_acc: 0.1333
Epoch 12/18
25s - loss: 2.2319 - acc: 0.2089 - val_loss: 2.2897 - val_acc: 0.1600
Epoch 13/18
25s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [15]:
def eight_model():
    # create model
    model = Sequential()
    model.add(Dense(145, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(51, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_8 = eight_model()

fit_8 = model_8.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69784,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_8 = model_8.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_8)))

Epoch 1/18
25s - loss: 2.4328 - acc: 0.1346 - val_loss: 2.4146 - val_acc: 0.0667
Epoch 2/18
25s - loss: 2.3592 - acc: 0.1630 - val_loss: 2.3653 - val_acc: 0.0933
Epoch 3/18
25s - loss: 2.3188 - acc: 0.1750 - val_loss: 2.3337 - val_acc: 0.0800
Epoch 4/18
25s - loss: 2.2979 - acc: 0.1845 - val_loss: 2.3126 - val_acc: 0.0933
Epoch 5/18
19s - loss: 2.2868 - acc: 0.1868 - val_loss: 2.3064 - val_acc: 0.0933
Epoch 6/18
18s - loss: 2.2734 - acc: 0.1931 - val_loss: 2.3121 - val_acc: 0.1067
Epoch 7/18
18s - loss: 2.2635 - acc: 0.1978 - val_loss: 2.3029 - val_acc: 0.1333
Epoch 8/18
18s - loss: 2.2584 - acc: 0.1981 - val_loss: 2.3103 - val_acc: 0.1067
Epoch 9/18
18s - loss: 2.2501 - acc: 0.2032 - val_loss: 2.3038 - val_acc: 0.1200
Epoch 10/18
18s - loss: 2.2422 - acc: 0.2045 - val_loss: 2.2970 - val_acc: 0.1467
Epoch 11/18
18s - loss: 2.2385 - acc: 0.2096 - val_loss: 2.3014 - val_acc: 0.1333
Epoch 12/18
18s - loss: 2.2289 - acc: 0.2108 - val_loss: 2.2996 - val_acc: 0.1600
Epoch 13/18
18s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [41]:
def ninth_model():
    # create model
    model = Sequential()
    model.add(Dense(156, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(51, input_dim=X_train.shape[1], init='normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(12, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

model_9 = ninth_model()

fit_9 = model_9.fit_generator(generator=batch_generator(X_train, y_train, 400, True),
                         nb_epoch=18,
                         samples_per_epoch=69884,
                         validation_data=(X_val.todense(), y_val), verbose=2
                         )

# evaluate the model

scores_val_9 = model_9.predict_generator(generator=batch_generatorp(X_val, 400, False), val_samples=X_val.shape[0])
print('logloss val {}'.format(log_loss(y_val, scores_val_9)))

Epoch 1/18
19s - loss: 2.4330 - acc: 0.1360 - val_loss: 2.4163 - val_acc: 0.0400
Epoch 2/18
19s - loss: 2.3586 - acc: 0.1622 - val_loss: 2.3704 - val_acc: 0.0800
Epoch 3/18
19s - loss: 2.3165 - acc: 0.1749 - val_loss: 2.3180 - val_acc: 0.1067
Epoch 4/18
19s - loss: 2.2984 - acc: 0.1828 - val_loss: 2.3054 - val_acc: 0.1200
Epoch 5/18
20s - loss: 2.2864 - acc: 0.1907 - val_loss: 2.3051 - val_acc: 0.1067
Epoch 6/18
20s - loss: 2.2721 - acc: 0.1927 - val_loss: 2.2936 - val_acc: 0.1067
Epoch 7/18
20s - loss: 2.2653 - acc: 0.1982 - val_loss: 2.3034 - val_acc: 0.1333
Epoch 8/18
20s - loss: 2.2557 - acc: 0.2013 - val_loss: 2.2867 - val_acc: 0.1333
Epoch 9/18
20s - loss: 2.2508 - acc: 0.2024 - val_loss: 2.3017 - val_acc: 0.1200
Epoch 10/18
20s - loss: 2.2422 - acc: 0.2075 - val_loss: 2.2989 - val_acc: 0.1467
Epoch 11/18
20s - loss: 2.2360 - acc: 0.2077 - val_loss: 2.2885 - val_acc: 0.1467
Epoch 12/18
20s - loss: 2.2290 - acc: 0.2134 - val_loss: 2.2892 - val_acc: 0.1200
Epoch 13/18
20s - loss: 2

/home/ubuntu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in double_scalars


In [17]:
print("# Final prediction")
scores_3 = model_3.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_3 = pd.DataFrame(scores_3 , columns=lable_group.classes_)
result_3["device_id"] = device_id
print(result_3.head(1))
result_3 = result_3.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-    F24-26    F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000138  0.000508  0.000948  0.003894  0.013173  0.020745  0.002387   

     M23-26    M27-28    M29-31    M32-38      M39+            device_id  
0  0.033445  0.044551  0.133506  0.369774  0.376932  1002079943728939269  


In [18]:
print("# Final prediction")
scores_2 = model_2.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_2 = pd.DataFrame(scores_2 , columns=lable_group.classes_)
result_2["device_id"] = device_id
print(result_2.head(1))
result_2 = result_2.set_index("device_id")

# Final prediction
       F23-    F24-26    F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000484  0.001221  0.002257  0.007976  0.034026  0.057715  0.003035   

     M23-26    M27-28    M29-31    M32-38      M39+            device_id  
0  0.015861  0.031093  0.083018  0.265599  0.497715  1002079943728939269  


In [19]:
print("# Final prediction")
scores = model.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result = pd.DataFrame(scores , columns=lable_group.classes_)
result["device_id"] = device_id
print(result.head(1))
result = result.set_index("device_id")

#result.to_csv('keras_' + str(log_loss(y_val, scores_val)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-   F24-26    F27-28    F29-32    F33-42     F43+      M22-  \
0  0.000035  0.00015  0.000347  0.002641  0.017033  0.02544  0.000846   

     M23-26    M27-28   M29-31   M32-38     M39+            device_id  
0  0.013455  0.022123  0.09382  0.31628  0.50783  1002079943728939269  


In [20]:
print("# Final prediction")
scores_4 = model_4.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_4 = pd.DataFrame(scores_4 , columns=lable_group.classes_)
result_4["device_id"] = device_id
print(result_4.head(1))
result_4 = result_4.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-    F24-26   F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000174  0.000609  0.00185  0.005094  0.024427  0.062238  0.002163   

     M23-26    M27-28   M29-31   M32-38      M39+            device_id  
0  0.018281  0.035111  0.09415  0.28475  0.471152  1002079943728939269  


In [21]:
print("# Final prediction")
scores_5 = model_5.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_5 = pd.DataFrame(scores_5 , columns=lable_group.classes_)
result_5["device_id"] = device_id
print(result_5.head(1))
result_5 = result_5.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-    F24-26    F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000274  0.000913  0.002163  0.006315  0.028079  0.050359  0.003889   

     M23-26    M27-28    M29-31    M32-38      M39+            device_id  
0  0.029594  0.039524  0.115268  0.260335  0.463286  1002079943728939269  


In [22]:
print("# Final prediction")
scores_6 = model_6.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_6 = pd.DataFrame(scores_6 , columns=lable_group.classes_)
result_6["device_id"] = device_id
print(result_6.head(1))
result_6 = result_6.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
      F23-    F24-26   F27-28    F29-32    F33-42     F43+      M22-  \
0  0.00015  0.000854  0.00177  0.007022  0.029162  0.06107  0.001486   

     M23-26    M27-28    M29-31    M32-38      M39+            device_id  
0  0.016423  0.031027  0.093113  0.247202  0.510721  1002079943728939269  


In [23]:
print("# Final prediction")
scores_7 = model_7.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_7 = pd.DataFrame(scores_7 , columns=lable_group.classes_)
result_7["device_id"] = device_id
print(result_7.head(1))
result_7 = result_7.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-    F24-26    F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000073  0.000406  0.001144  0.003903  0.021798  0.041399  0.000882   

     M23-26    M27-28    M29-31    M32-38      M39+            device_id  
0  0.017247  0.029377  0.091626  0.240454  0.551691  1002079943728939269  


In [24]:
print("# Final prediction")
scores_8 = model_8.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_8 = pd.DataFrame(scores_8 , columns=lable_group.classes_)
result_8["device_id"] = device_id
print(result_8.head(1))
result_8 = result_8.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-    F24-26    F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000344  0.001679  0.003131  0.008121  0.027526  0.054043  0.004425   

     M23-26    M27-28   M29-31   M32-38      M39+            device_id  
0  0.035263  0.057595  0.12862  0.26707  0.412184  1002079943728939269  


In [42]:
print("# Final prediction")
scores_9 = model_9.predict_generator(generator=batch_generatorp(test_sp, 800, False), val_samples=test_sp.shape[0])
result_9 = pd.DataFrame(scores_9 , columns=lable_group.classes_)
result_9["device_id"] = device_id
print(result_9.head(1))
result_9 = result_9.set_index("device_id")

#result.to_csv('./sub_bagofapps7_keras_10_50_pt2_10epoch.csv', index=True, index_label='device_id')
#Drop out 0.2
#Validation 2.3017
#result_3.to_csv('keras_' + str(log_loss(y_val, scores_val_3)) + '.csv', index=True, index_label='device_id')

# Final prediction
       F23-    F24-26    F27-28    F29-32    F33-42      F43+      M22-  \
0  0.000141  0.000402  0.000939  0.004165  0.025532  0.049513  0.001885   

     M23-26    M27-28    M29-31    M32-38      M39+            device_id  
0  0.013404  0.024059  0.069278  0.205052  0.605631  1002079943728939269  


In [47]:
#tot = result + result_2 + result_3 + result_4 + result_5 + result_6
#tot = result_3 + result_5 + result_6 
#tot = result + result_2 + result_3 + result_4 + result_5 + result_6 + result_7
#tot = result + result_2 + result_3 + result_4 + result_5 + result_6 + result_7 + result_8 + result_9
tot = result + result_2 + result_3 + result_5 + result_6 + result_7 + result_8 + result_9

In [48]:
#tot = tot/6
#tot = tot/3
#tot = tot/7
tot = tot/8
#tot = tot/10

In [49]:
tot[tot<0.001]=0.001 # 1,2,3,4,5,5 models + this -> 2.23982
tot[tot>0.999]=0.999 # 1,2,3,4,5,5 models + this -> 2.23982

In [ ]:
#tot.to_csv('keras_' + str(log_loss(y_val, (scores_val + scores_val_2 + scores_val_3 + scores_val_4 + scores_val_5 + scores_val_6)/6)) + '.csv', index=True, index_label='device_id')

In [ ]:
tot.to_csv('keras_' + str(log_loss(y_val, (scores_val_6 + scores_val_3 + scores_val_5)/3)) + '.csv', index=True, index_label='device_id')

In [50]:
tot.to_csv('keras_' + str(log_loss(y_val, (scores_val + scores_val_2 + scores_val_3 + scores_val_5 + scores_val_6 + scores_val_7 + scores_val_8 + scores_val_9)/8)) + '.csv', index=True, index_label='device_id')

In [46]:
tot.to_csv('keras_' + str(log_loss(y_val, (scores_val + scores_val_2 + scores_val_3 + scores_val_4 + scores_val_5 + scores_val_6 + scores_val_7 + scores_val_8 + scores_val_9)/9)) + '.csv', index=True, index_label='device_id')